In [1]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, asin, sqrt, pi, atan, atan2, fabs
import time

In [4]:
def calcular_distancia(lat1,long1,lat2,long2):
    lon1, lat1 = (radians(coord) for coord in (long1,lat1))
    lon2, lat2 = (radians(coord) for coord in (long2,lat2))
    dlat = (lat2 - lat1)
    dlon = (lon2 - lon1)
    a = (
        sin(dlat * 0.5)**2 +
        cos(lat1) * cos(lat2) * sin(dlon * 0.5)**2
    )
    
    radioTierra = 6371008.8
    return 2 * radioTierra * asin(sqrt(a)) #lo dejo en metros

#distancia de Haversine
#https://gist.github.com/habibutsu/8bbcc202a915e965c6a6d4f561d0e482

In [3]:
propiedades = pd.read_csv('/home/mati/Desktop/properati.csv')
#propiedades = pd.read_csv('/home/agustin/Escritorio/escritorio/fiuba/Organizacion de datos/properati.csv')

In [5]:
propiedades = propiedades.loc[propiedades.Year >= 2016,:]

In [6]:
propiedades = propiedades.loc[(propiedades.lat.notnull()) & (propiedades.lon.notnull()) &\
                             (propiedades.superficie.notnull()) & (propiedades.price_aprox_usd.notnull()),:]

## Tomo como set de prueba las publicaciones de junio 2017

In [22]:
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),:].reset_index()
set_datos = propiedades.loc[(propiedades.Year != 2017) | (propiedades.Month < 6),:].reset_index()

In [23]:
set_pruebas = set_pruebas.sample(frac = 0.12).reset_index()
set_pruebas.loc[:,'resultado'] = 0
set_pruebas.loc[:,'cantidad_analizados'] = 0
set_pruebas = set_pruebas.loc[:,['property_type','lat','lon','superficie','price_aprox_usd','resultado','cantidad_analizados']]
set_pruebas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7519 entries, 0 to 7518
Data columns (total 7 columns):
property_type          7519 non-null object
lat                    7519 non-null float64
lon                    7519 non-null float64
superficie             7519 non-null float64
price_aprox_usd        7519 non-null float64
resultado              7519 non-null int64
cantidad_analizados    7519 non-null int64
dtypes: float64(4), int64(2), object(1)
memory usage: 411.3+ KB


In [25]:
set_datos = set_datos.loc[:,['property_type','lat','lon','superficie','price_aprox_usd']]
set_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464833 entries, 0 to 464832
Data columns (total 5 columns):
property_type      464833 non-null object
lat                464833 non-null float64
lon                464833 non-null float64
superficie         464833 non-null float64
price_aprox_usd    464833 non-null float64
dtypes: float64(4), object(1)
memory usage: 17.7+ MB


### Agrupo por tipo de propiedad y luego por superficie, serian los lideres de knn

In [26]:
def redondear_superficie(superficie):
    """Convierto la superficie a un multiplo de 10"""
    superficie /= 10
    superficie = round(superficie) #lo hago entero
    superficie *= 10
    return int(superficie)

In [27]:
set_datos.loc[:,'superficie_actualizada'] = set_datos.loc[:,'superficie'].apply(redondear_superficie)
set_datos.head(2)

,property_type,lat,lon,superficie,price_aprox_usd,superficie_actualizada
0,house,-34.603282,-58.525425,448.0,490000.0,450
1,apartment,-34.564741,-58.441372,308.0,810000.0,310


In [29]:
set_datos_agrupados = set_datos.groupby('property_type')

In [30]:
#Agrupo por superficie cada grupo
grupos = {}
for grupo in set_datos_agrupados.__iter__():
    clave = grupo[0]
    valor = grupo[1].groupby('superficie_actualizada')
    diccionario = {}
    for group in valor.__iter__():
        sub_clave = group[0]
        sub_valor = group[1]
        diccionario[sub_clave] = sub_valor
    grupos[clave] = diccionario

In [31]:
set_datos_agrupados = grupos

#### Los datos quedan dic = {prop_type: {sup : dataframe}}

## Algoritmo de KNN

In [32]:
def calcular_distancias_para_knn(datos,prueba_type,prueba_sup,prueba_lat,prueba_lon):
    grupo = datos[prueba_type]
    prueba_sup = redondear_superficie(prueba_sup)
    analizados = []
    for sup in range(prueba_sup - 10, prueba_sup + 20, 10):  #Miro sup +- 10
        try:
            a_analizar = grupo[sup].reset_index()
            for i in range(len(a_analizar)):
                distancia = calcular_distancia(a_analizar.lat[i],a_analizar.lon[i],prueba_lat,prueba_lon)
                analizados.append((distancia,a_analizar.price_aprox_usd[i]))
        except:
            #Si no hay ninguna propiedad con superficie sup
            continue
    return analizados

In [33]:
def knn(datos,pruebas,k):
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.property_type[i],pruebas.superficie[i],\
                                                  pruebas.lat[i],pruebas.lon[i])
        resultado = obtener_promedio_k_minimos(distancias,k)
        pruebas.resultado[i] = resultado
        pruebas.cantidad_analizados[i] = len(distancias)
    return pruebas

In [56]:
def knn_con_precision(datos,pruebas,lista_k): #Para buscar el mejor K de una lista
    resultados = {}
    for i in range(len(pruebas)):
        distancias = calcular_distancias_para_knn(datos,pruebas.property_type[i],pruebas.superficie[i],\
                                                  pruebas.lat[i],pruebas.lon[i])
        resultados = obtener_promedio_k_minimos(resultados,pruebas.price_aprox_usd[i],distancias,lista_k)
    return resultados

In [55]:
def obtener_promedio_k_minimos(resultados,real,distancias,lista_k):
    distancias.sort()
    for k in lista_k:
        k = min(k,len(distancias))
        a_analizar = distancias[:k]
        suma = 0
        total = 0
        for x in range(len(a_analizar)):
            suma += a_analizar[x][1]
            total += 1
        res = suma / total
        lista = resultados.get(k,[])
        lista.append((real,res))
        resultados[k] = lista
    return resultados

In [44]:
def calcular_precision(resultados):
    res = {}
    for k in resultados.keys():
        suma = 0
        total = 0
        lista = resultados[k]
        for tupla in lista:
            suma += calcular_resultado(tupla[0],tupla[1])
            total += 1
        res[k] = round(suma*100 / total,2)
    return res

In [58]:
def calcular_resultado(real,calculado):
    porcentaje = 0.025
    if abs(real - calculado) <= real * porcentaje:
        return 1
    return 0

## Busqueda del hiperparametro K

##### Empezamos con algunos k salteados

In [57]:
lista_k = [5,10,20,30,50,100,200,350,500,750,1000,2000,3000,10000,20000,50000,100000]

inicio = time.strftime("%H:%M:%S")
pruebas = knn_con_precision(set_datos_agrupados,set_pruebas.head(2),lista_k)
precision = calcular_precision(pruebas)
print("Inicio: ({}) Final: ({})\n".format(inicio,time.strftime("%H:%M:%S")))
for k in lista_k:
    print("Para k = {} la precision es {}%".format(k,precision[k]))


157000.0 156410.0
98000.0 89900.0
157000.0 150595.0
98000.0 98025.0
Inicio: (11:27:32) Final: (11:28:55)
Para k = 30 la precision es 50.0%
Para k = 40 la precision es 50.0%
